<a href="https://colab.research.google.com/github/samuelj7/big-data-challenge/blob/main/Big_Data_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz"

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
#Make connection to postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in .csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

380604
380575
380575


In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col

In [ ]:
from pyspark.sql.functions import to_date

In [ ]:
df_v2 = df.withColumn('review_date', to_date('review_date', 'yyyy-MM-dd'))
df_v2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   39107297|R1000G235G84VS|6304428456|     619227014|      Daylight [VHS]|           Video|          5|            0|          7|   N|                N|              laneme|fast shipping tap...| 2002-07-30|
|         US|   46346809|R103NDNKDVKGYD|B00AVPYKFU|     631259326|Blank VHS Tape So...|           Video|          5|    

In [ ]:
df_v2.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
df_v3 = df_v2.drop("marketplace", "product_category", "verified_purchase", "review_headline", "review_body")
df_v3.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   39107297|R1000G235G84VS|6304428456|     619227014|      Daylight [VHS]|          5|            0|          7|   N| 2002-07-30|
|   46346809|R103NDNKDVKGYD|B00AVPYKFU|     631259326|Blank VHS Tape So...|          5|            0|          0|   N| 2015-04-24|
|   42564085|R1068F9HHXTYR5|6304898592|     478267204|Baby's First Impr...|          1|            0|          0|   N| 2004-09-17|
|   46499989|R10D7QYJVUDBJI|1574924494|     716139900|   The Odyssey [VHS]|          3|            0|          1|   N| 2001-01-02|
|   50133272|R10HY4HIY7HLIY|6301980506|     557595408|Strike Up the Ban...|        

In [ ]:
df_v3.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
#Customer Table

df_v4 = df_v3.groupBy("customer_id").count()
customer=df_v4.withColumnRenamed("count", "customer_count")
customer.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   20288432|             2|
|   49145342|             2|
|   21187650|             2|
|   27170594|             1|
|   36395157|             5|
|   41711141|             1|
|   33782391|             1|
|   12491275|             1|
|   49501007|             2|
|   51667424|            61|
|   28425084|             1|
|   52119549|             1|
|   36728400|             1|
|   53033948|            15|
|   32371281|             4|
|   11736558|             2|
|   42929754|             1|
|   51680053|            48|
|   50900268|             1|
|   43293705|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Review ID Table
review_id_table = df_v3.select(["review_id", "customer_id", "product_id", "product_parent", "review_date" ])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000G235G84VS|   39107297|6304428456|     619227014| 2002-07-30|
|R103NDNKDVKGYD|   46346809|B00AVPYKFU|     631259326| 2015-04-24|
|R1068F9HHXTYR5|   42564085|6304898592|     478267204| 2004-09-17|
|R10D7QYJVUDBJI|   46499989|1574924494|     716139900| 2001-01-02|
|R10HY4HIY7HLIY|   50133272|6301980506|     557595408| 2009-11-15|
|R10NGF2CDJZBND|   51803143|B00007M5KS|     544248946| 2004-04-22|
|R10OUGP1VVK7JG|   44872476|6301906500|     103981020| 2005-09-21|
|R10OVB2XLAB5YA|   49936699|B00005PJ7L|      86419093| 2001-10-29|
|R10S1BI2PXII2T|   52049485|6303925332|     812288877| 2004-04-13|
|R10TIKWPEVXNG7|   50205849|B00005BCNZ|     908216169| 2006-11-15|
|R10VND2YDQVKET|   52415749|B00000FEJW|     324641956| 2001-06-11|
|R10X37FJGANCT7|   51962520|0792152328|     764225016| 1999-10

In [ ]:
#Products Table
products = df_v3.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|6304428456|      Daylight [VHS]|
|B00AVPYKFU|Blank VHS Tape So...|
|6304898592|Baby's First Impr...|
|1574924494|   The Odyssey [VHS]|
|6301980506|Strike Up the Ban...|
|B00007M5KS|The Gathering Sto...|
|6301906500|The Herculoids, V...|
|B00005PJ7L|Erotic Rites of C...|
|6303925332|Timon & Pumbaa: G...|
|B00005BCNZ| Mystery Ranch [VHS]|
|B00000FEJW|Troublesome Creek...|
|0792152328|       Titanic [VHS]|
|0790737442|Lethal Weapon 2 [...|
|B0000V46RW|    Wrong Turn [VHS]|
|B00000F921|   Straight Up [VHS]|
|6304493053|Incredible Journe...|
|B000006D2X|      Quilombo [VHS]|
|6303379133|The Iliad (Homer)...|
|B0002CDM7E|The Original 29 M...|
|6303451438|Knock on Any Door...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
#Vine Table
vine = df_v3.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine" ])
vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000G235G84VS|          5|            0|          7|   N|
|R103NDNKDVKGYD|          5|            0|          0|   N|
|R1068F9HHXTYR5|          1|            0|          0|   N|
|R10D7QYJVUDBJI|          3|            0|          1|   N|
|R10HY4HIY7HLIY|          5|            0|          0|   N|
|R10NGF2CDJZBND|          5|           61|         63|   N|
|R10OUGP1VVK7JG|          4|           10|         10|   N|
|R10OVB2XLAB5YA|          5|            2|         12|   N|
|R10S1BI2PXII2T|          4|            1|          2|   N|
|R10TIKWPEVXNG7|          5|            0|          0|   N|
|R10VND2YDQVKET|          5|            2|          2|   N|
|R10X37FJGANCT7|          1|            7|         13|   N|
|R10YNLQ6Y1R161|          5|            0|          1|   N|
|R110565ZKI140J|          4|            

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"postgres",
          "password": "<postgres>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table (Vine)

vine.write.jdbc(url=jdbc_url, table="vine", mode=mode, properties=config)

In [ ]:
# Write  DataFrame to table (Customer)


customer.write.jdbc(url=jdbc_url, table="customer", mode=mode, properties=config)

In [ ]:
# Write  DataFrame to table (Review ID)


review_id_table.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write  DataFrame to table (Products)


products.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)